# Continuing from our N3 Ontology...  

Let's build the same-ish thing in **OWL, RDF and Turtle** - these are various formats for represnting Ontologies.

To review:

# WHAT

1. **What are the “things” (nouns) we care about?**
   * e.g. *Movie*, *Actor*, *Director*, *Character*, *Genre*

2. **What are their relationships to each other (verbs)?**
   * *directed by, acted in, plays, belongs to, has title, has release date*

3. **What data (literals) do we need?**
   * *title (string), releaseDate (date), birthDate (date), runtime (integer)*

4. **What constraints make sense?**
   * Every Movie must have at least one Director.
   * A Character must be played by exactly one Actor.

# Just OWL first

## Ontology concepts:

* **Classes** (`owl:Class`): types of things.
* **ObjectProperties** (`owl:ObjectProperty`): links between individuals.
* **DatatypeProperties** (`owl:DatatypeProperty`): links to literal values.
* **Restrictions** (`owl:Restriction`): e.g. cardinality or value constraints.
* **Inverse properties** (`owl:inverseOf`): auto-generate reverse links.
* **Rules** (in N3) using the `log:` vocabulary for simple inference.

# Our first lines of code  

### Prefix Declarations

```
Prefix(:=<http://example.org/movies#>)
Prefix(owl:=<http://www.w3.org/2002/07/owl#>)
```

Map a short label to a long IRI so you can write `:Movie` instead of the full `<http://…#Movie>`.


### Ontology Declaration

```
Ontology(<http://example.org/movies> …)
```

Gives the ontology its own IRI (its identifier).

### Class and Property Declarations

* `Declaration(Class(\:Movie))`  introduces `:Movie` as a class.
* `AnnotationAssertion(rdfs\:label \:Movie "Movie")`  adds a human-readable label.
* `Declaration(ObjectProperty(\:directedBy))`  introduces `:directedBy` as an object property.
* `ObjectPropertyDomain(\:directedBy \:Movie)`  restricts the subject of `:directedBy` to instances of `:Movie`.
* `ObjectPropertyRange(\:directedBy \:Director)`  restricts the object of `:directedBy` to instances of `:Director`.

### Datatype Properties

Analogous to object properties but link to literals:

```
Declaration(DatatypeProperty(:releaseDate))
DatatypePropertyRange(:releaseDate xsd:date)
```

### Inverse Properties

```
ObjectPropertyInverseOf(:directs :directedBy)
```

Declares `:directs` as the inverse of `:directedBy`.

### Class Restrictions

```
SubClassOf(
  :Movie
  ObjectMinCardinality(1 :directedBy :Director)
)
```

Says every `:Movie` must have at least one `:directedBy` link to a `:Director`.

# Output

```owl
Prefix(:=<http://example.org/movies#>)
Prefix(owl:=<http://www.w3.org/2002/07/owl#>)
Prefix(rdfs:=<http://www.w3.org/2000/01/rdf-schema#>)
Prefix(xsd:=<http://www.w3.org/2001/XMLSchema#>)

Ontology(<http://example.org/movies>

  ### Classes ###
  Declaration(Class(:Movie))
  AnnotationAssertion(rdfs:label :Movie "Movie")

  Declaration(Class(:Actor))
  AnnotationAssertion(rdfs:label :Actor "Actor")

  Declaration(Class(:Director))
  AnnotationAssertion(rdfs:label :Director "Director")

  Declaration(Class(:Character))
  AnnotationAssertion(rdfs:label :Character "Character")

  Declaration(Class(:Genre))
  AnnotationAssertion(rdfs:label :Genre "Genre")

  ### Object Properties ###
  Declaration(ObjectProperty(:directedBy))
  AnnotationAssertion(rdfs:label :directedBy "directed by")
  ObjectPropertyDomain(:directedBy :Movie)
  ObjectPropertyRange(:directedBy :Director)

  Declaration(ObjectProperty(:directs))
  ObjectPropertyInverseOf(:directs :directedBy)
  ObjectPropertyDomain(:directs :Director)
  ObjectPropertyRange(:directs :Movie)

  Declaration(ObjectProperty(:actedIn))
  AnnotationAssertion(rdfs:label :actedIn "acted in")
  ObjectPropertyDomain(:actedIn :Actor)
  ObjectPropertyRange(:actedIn :Movie)

  Declaration(ObjectProperty(:characterPlayedBy))
  Declaration(ObjectProperty(:playsCharacter))
  AnnotationAssertion(rdfs:label :playsCharacter "plays character")
  ObjectPropertyInverseOf(:playsCharacter :characterPlayedBy)
  ObjectPropertyDomain(:playsCharacter :Actor)
  ObjectPropertyRange(:playsCharacter :Character)

  ### Datatype Properties ###
  Declaration(DatatypeProperty(:title))
  AnnotationAssertion(rdfs:label :title "title")
  DatatypePropertyDomain(:title :Movie)
  DatatypePropertyRange(:title xsd:string)

  Declaration(DatatypeProperty(:releaseDate))
  AnnotationAssertion(rdfs:label :releaseDate "release date")
  DatatypePropertyDomain(:releaseDate :Movie)
  DatatypePropertyRange(:releaseDate xsd:date)

  ### Class Restrictions ###
  SubClassOf(
    :Movie
    ObjectMinCardinality(1 :directedBy :Director)
  )

  SubClassOf(
    :Character
    ObjectExactCardinality(1 :characterPlayedBy :Actor)
  )

)
```

# RDF, OWL and Turtle  
This is where it gets interesting

* **RDF** is the underlying data model: triples of the form *subject–predicate–object*.
* **RDFS** (RDF Schema) is an extension vocabulary on top of RDF that provides basic ontology constructs:
    * `rdfs:Class`, `rdfs:subClassOf`, `rdfs:label`, etc.
    * It lets you define simple hierarchies and metadata about your terms.
* **OWL** (Web Ontology Language) is built on RDF/RDFS and adds richer modelling features:
    * Classes, properties, and highly expressive constraints (cardinalities, boolean class constructors, property characteristics, etc.).
    * Terms live in the `owl:` namespace, e.g. `owl:Class`, `owl:ObjectProperty`, `owl:Restriction`.
* **Turtle** is merely a **syntax** for writing down RDF graphs (including RDFS- or OWL-based ontologies) in a compact, human-readable form.
    * It defines how to express prefixes, triples, blank nodes, literals, collections, etc.

## Core data model (`rdf:`)

Use RDF terms when you’re just expressing the basic fact-triple structure or need the fundamental “typing” mechanism:

* **`rdf:type`**
  – “This thing is an instance of that class.”
* **`rdf:Property`**
  – If you need to declare a generic property (before you decide if it’s an object vs. datatype property).
* **Containers & collections** (`rdf:List`, `rdf:first`, `rdf:rest`)
  – Only when you need ordered or grouped lists.

**Intuition:** “we just need to say A → B → C in RDF.”

## Basic schema‐level terms (`rdfs:`)

Use RDFS to give structure and human‐friendly metadata:

* **`rdfs:Class`**
  – Define a category of things.
* **`rdfs:subClassOf`**
  – Build simple hierarchies (e.g. `:ActionMovie` ⊆ `:Movie`).
* **`rdfs:domain`** / **`rdfs:range`**
  – Roughly constrain which subjects/objects a property can take.
* **`rdfs:label`** / **`rdfs:comment`**
  – Add a readable name or description.

**Intuition:** “we want minimal typing and basic hierarchies, plus labels so people know what my terms mean.”

## Rich ontology features (`owl:`)

Reach for OWL when you need expressive constraints and formal semantics:

* **`owl:Class`** vs. `rdfs:Class`
  – OWL classes participate in richer modeling (boolean combinations, disjointness).
* **`owl:ObjectProperty`** / **`owl:DatatypeProperty`**
  – Clearly distinguish links to resources vs. literals.
* **Property characteristics**
  – `owl:FunctionalProperty`, `owl:InverseFunctionalProperty`, `owl:TransitiveProperty`, etc.
* **Restrictions & cardinalities**
  – `owl:onProperty`, `owl:minCardinality`, `owl:allValuesFrom`, …
* **Inverse & symmetric properties**
  – `owl:inverseOf`, `owl:SymmetricProperty`
* **Class constructors**
  – `owl:unionOf`, `owl:intersectionOf`, `owl:complementOf`
* **Assertions about individuals** (in OWL DL/RDF you often declare individuals of classes, sameAs, differentFrom, etc.)

**Intuition:** “we need full logical rigor—exact counts, complex class definitions, property rules—so a reasoner can validate and infer over my model.”

## decision flow

1. **Just facts & typing?** → use **`rdf:`**
2. **Simple schema & labels?** → add **`rdfs:`**
3. **Formal constraints & inference?** → use **`owl:`**

# Building the Ontology (just like before...)

##  Prefixes

At the top of a Turtle file you bind prefixes to IRIs so you can write `:` instead of the full IRI:

```turtle
@prefix :     <http://example.org/movies#> .
@prefix owl:  <http://www.w3.org/2002/07/owl#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd:  <http://www.w3.org/2001/XMLSchema#> .
```

* **`@prefix name: <IRI> .`** declares a namespace.

##  Declaring Classes

A class is an individual of `owl:Class`. We give it a label for readability:

```turtle
:Movie    a owl:Class ;
    rdfs:label "Movie" .

:Actor    a owl:Class ;
    rdfs:label "Actor" .
```

* **`subject predicate object .`** is the core triple form.
* **`a`** is shorthand for `rdf:type`.
* **`;`** continues more predicates on the same subject.

##  Object and Datatype Properties

We declare each property, its domain and range:

```turtle
:directedBy
    a       owl:ObjectProperty ;
    rdfs:domain :Movie ;
    rdfs:range  :Director ;
    rdfs:label  "directed by" .

:releaseDate
    a       owl:DatatypeProperty ;
    rdfs:domain :Movie ;
    rdfs:range  xsd:date ;
    rdfs:label  "release date" .
```

* **`owl:ObjectProperty`** vs **`owl:DatatypeProperty`**
* **`rdfs:domain`** limits subjects; **`rdfs:range`** limits objects.

##  Inverse Properties

Use `owl:inverseOf` to link two properties as inverses:

```turtle
:directs
    a               owl:ObjectProperty ;
    owl:inverseOf   :directedBy ;
    rdfs:domain     :Director ;
    rdfs:range      :Movie ;
    rdfs:label      "directs" .
```

* Inference engines will know that if `:d directs :m` then `:m directedBy :d`.

##  Restrictions via Blank Nodes

To constrain a class, attach an anonymous restriction node with `rdfs:subClassOf`:

```turtle
:Movie
    rdfs:subClassOf [
        a                 owl:Restriction ;
        owl:onProperty    :directedBy ;
        owl:minCardinality "1"^^xsd:nonNegativeInteger
    ] .
```

* **`[ … ]`** denotes a blank (anonymous) node.
* **`owl:onProperty`** picks the property being restricted.
* **`owl:minCardinality`, `owl:maxCardinality`, `owl:cardinality`, `owl:someValuesFrom`, `owl:allValuesFrom`** are the main restriction types.
* Literals like `"1"^^xsd:nonNegativeInteger` use `^^` to type the value.

##  Complete Turtle Ontology

```turtle
@prefix :     <http://example.org/movies#> .
@prefix owl:  <http://www.w3.org/2002/07/owl#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd:  <http://www.w3.org/2001/XMLSchema#> .

# Classes
:Movie
    a           owl:Class ;
    rdfs:label  "Movie" ;
    rdfs:subClassOf [
        a                 owl:Restriction ;
        owl:onProperty    :directedBy ;
        owl:minCardinality "1"^^xsd:nonNegativeInteger
    ] .

:Actor
    a           owl:Class ;
    rdfs:label  "Actor" .

:Director
    a           owl:Class ;
    rdfs:label  "Director" .

:Character
    a           owl:Class ;
    rdfs:label  "Character" ;
    rdfs:subClassOf [
        a               owl:Restriction ;
        owl:onProperty  :characterPlayedBy ;
        owl:cardinality "1"^^xsd:nonNegativeInteger
    ] .

:Genre
    a           owl:Class ;
    rdfs:label  "Genre" .

# Object Properties
:directedBy
    a           owl:ObjectProperty ;
    rdfs:domain :Movie ;
    rdfs:range  :Director ;
    rdfs:label  "directed by" .

:directs
    a               owl:ObjectProperty ;
    owl:inverseOf   :directedBy ;
    rdfs:domain     :Director ;
    rdfs:range      :Movie ;
    rdfs:label      "directs" .

:actedIn
    a           owl:ObjectProperty ;
    rdfs:domain :Actor ;
    rdfs:range  :Movie ;
    rdfs:label  "acted in" .

:characterPlayedBy
    a           owl:ObjectProperty ;
    rdfs:domain :Character ;
    rdfs:range  :Actor .

:playsCharacter
    a               owl:ObjectProperty ;
    owl:inverseOf   :characterPlayedBy ;
    rdfs:domain     :Actor ;
    rdfs:range      :Character ;
    rdfs:label      "plays character" .

# Datatype Properties
:title
    a           owl:DatatypeProperty ;
    rdfs:domain :Movie ;
    rdfs:range  xsd:string ;
    rdfs:label  "title" .

:releaseDate
    a           owl:DatatypeProperty ;
    rdfs:domain :Movie ;
    rdfs:range  xsd:date ;
    rdfs:label  "release date" .
```